# Generate adjacency matrix PKL file

In [1]:
import torch
import pickle, gzip
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import numpy as np
import pandas as pd
import os
from voc import read_object_labels_csv

/home/seongha//envs/LTML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coco_lt_path = "data/coco"
voc_lt_path = "data/voc/"

# VOC2007

In [3]:
path_csv = '../data/voc/files/VOC2007'
trainval_csv = os.path.join(path_csv, 'classification_trainval.csv')
test_csv = os.path.join(path_csv, 'classification_test.csv')

In [4]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)

Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(5011, 21)


In [5]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)
gt_labels = hi.iloc[:,1:].to_numpy()
print(gt_labels.shape)
img_id2idx = dict()
idx2img_id = dict()
print(gt_labels)

for i, row in hi.iterrows():
  img_id2idx[row['name']] = i
  idx2img_id[i] = row['name']


Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(5011, 21)
(5011, 20)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id
}
with open('test_terse_gt_2007.pkl', 'wb') as f:
  pickle.dump(res, f)

# Generate VOC ADJ PKL file


corelation

In [7]:
## count number of correlations
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)

hi=hi.iloc[:,1:]
di = dict()
adj = []
nums = []
from collections import defaultdict
cnt  = defaultdict(int)
for col_name, val in hi.iteritems():
  if col_name =='name':
    continue
  print("colname: ",col_name)
  df = hi[hi[col_name].isin([1.0])]
  print(df.shape)
  for col_name_df, val_df in df.iteritems():
    if col_name_df == col_name:
      continue
    print(col_name_df)
    if val_df.value_counts().get(1):
      k = [col_name, col_name_df]
      k.sort()
      k = tuple(k)
      cnt[k] += val_df.value_counts().get(1)



cnt = {k:v for k, v in sorted(cnt.items(), key=lambda item: item[1], reverse=True)}
print(len(cnt), sum(cnt.values()) )
max(cnt.values()), min(cnt.values())



Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(5011, 21)
colname:  aeroplane
(240, 20)
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
colname:  bicycle
(255, 20)
aeroplane
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
colname:  bird
(333, 20)
aeroplane
bicycle
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
colname:  boat
(188, 20)
aeroplane
bicycle
bird
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
colname:  bottle
(262, 20)
aeroplane
bicycle
bird
boat
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
potted

(552, 2)

In [20]:
for k,v in cnt.items():
  
  print("{},{}".format(k[0],k[1]), v)

voc_joint = pd.DataFrame({"k0": [k[0] for k in cnt.keys()],
    "k1" : [k[1] for k in cnt.keys()],
    "v" : cnt.values()
})

voc_joint.to_csv("voc_joint.csv")

In [ ]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)

hi=hi.iloc[:,1:]
di = dict()
adj = []
nums = []
from collections import Counter
cnt  = Counter()
for col_name, val in hi.iteritems():
  # print(col_name, hi[hi[col_name].isin([1.0])].count().to_)
  if col_name =='name':
    continue
  # di[col_name] = val.value_counts().to_dict()
  M = hi[hi[col_name].isin([1.0])].sum(axis=0)
  # print(M)
  nums.append(M[col_name])
  # M[col_name] = 0
  # print(cond_prob[col_name])
  # print(M)
  print(sum(M.to_list()))
  adj.append(M.to_list())
  
  # print(hi[hi[col_name].isin([1.0])].sum(axis=0))
# nums.sort()
print(nums)
print(max(nums), min(nums))

adj = np.asarray(adj)
nums = np.asarray(nums)


di['adj'] = adj
di['nums'] = nums
print(di['adj'])

import json
with open("voc_class_num.json", 'w') as f:
  json.dump(di, f)


In [ ]:
import pickle
with open('voc_lt_adj.pkl', 'wb') as f:
  pickle.dump(di, f)

In [27]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)

hi=hi.iloc[:,1:]
di = dict()
adj = []
nums = []
from collections import defaultdict
cnt  = defaultdict(int)


Counter(hi.astype(bool).sum(axis=1).to_dict())
item_lab_cnt = hi.astype(bool).sum(axis=1).to_dict()
for k, v in item_lab_cnt.items():
  cnt[v] += 1

cnt

pd.DataFrame({
  
})

Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(5011, 21)


defaultdict(int, {1: 2808, 2: 1644, 3: 443, 4: 98, 5: 13, 6: 4, 7: 1})

In [8]:
##check
# print(nums)
nums2 = nums[:, np.newaxis]
# print(nums2)
# P = adj / nums2
P = np.divide(adj, nums2)
# print("p", P)

t = np.mean(P, axis=0)
np.set_printoptions(3)
# print("t:", t)
P[P < t] = 0 #balance
# print(P)
P[P >= t] = 1
print(P)
P = P * 0.2 / (P.sum(0, keepdims=True) + 1e-6) #reweight
print(P)
P = P + np.identity(len(nums), np.int) * 0.8
print(P)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.

/tmp/ipykernel_40963/1120479978.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  P = P + np.identity(len(nums), np.int) * 0.8


In [21]:
with open(voc_lt_path + "/testlongtail2007/img_id.txt", "r") as f:
  # data= pickle.load(f)
  data = f.readlines()
  # print(data)
  data = [int(each) for each in data]
  # data = list(map(str,data))
  print(len(data))
print(max(data))
# print(hi['name'].isin({'name': data}))
# print(hi['name'].dtype)

LT = hi.loc[hi['name'].isin(data)]

LT['name'] = LT['name'].apply(lambda x:format(x, '06d'))
print(LT['name'])
import os
LT_path = os.path.join(path_csv, 'classification_test_' + "LT.csv")
LT.to_csv(LT_path, index=False) 

1366
9963
0       000001
2       000003
3       000004
4       000006
5       000008
         ...  
4941    009941
4944    009951
4948    009957
4950    009962
4951    009963
Name: name, Length: 1366, dtype: object


/tmp/ipykernel_40963/2512746191.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LT['name'] = LT['name'].apply(lambda x:format(x, '06d'))


# COCO2014

In [28]:
path_csv = '../data/coco/'
import json

In [13]:

with open(path_csv+ '/data/category.json') as f:

  category = json.load(f)
  print(category.keys())
  print(len(category.items()))
  print(category.values())

with open(path_csv + '/data/train_anno.json') as f:

  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i
  print(max(li))

  df = pd.DataFrame(gt_labels)
  print(df.columns)
  di = dict()
  for col_name, val in df.iteritems():
    if col_name == 'name': continue
    print(val)
    df[col_name] = val.value_counts().to_dict()
  
  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

with open(path_csv + '/data/val_anno.json') as f:

  val = json.load(f)
  print(len(val), val[0])

  test_gt_labels = np.zeros((len(val),80))
  
  for i,each in enumerate(val):
    test_gt_labels[i, each['labels']] = 1
  print(test_gt_labels)

res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id,
  "test_gt_labels": test_gt_labels
}

with open('terse_gt_2014.pkl', 'wb') as f:
  pickle.dump(res, f)

dict_keys(['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra'])
80
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3

ValueError: Length of values (2) does not match length of index (82081)

Explore COCO-MLT

In [29]:
with open(path_csv + '/data/train_anno.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i
  nums = gt_labels.sum(axis=0)
  adj = []
  for i,col in enumerate(gt_labels.T):
    cond_prob = gt_labels[np.isin(col,[1.0]),:].sum(axis=0)
    cond_prob[i] = 0
    adj.append(cond_prob)
    # print(adj[-1])
  print(list(nums))
  nums.sort()
  print(max(nums), min(nums))
  di={'adj': np.asarray(adj), "nums": np.asarray(nums)}


82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
[2243.0, 1171.0, 3924.0, 1618.0, 1804.0, 1884.0, 668.0, 2539.0, 3844.0, 2287.0, 2241.0, 2098.0, 3734.0, 5968.0, 5028.0, 1340.0, 2791.0, 2080.0, 8606.0, 1186.0, 2818.0, 3322.0, 8950.0, 3159.0, 3170.0, 1389.0, 6518.0, 8378.0, 3041.0, 1062.0, 1518.0, 1205.0, 2537.0, 1511.0, 1798.0, 128.0, 4861.0, 2068.0, 821.0, 1471.0, 1625.0, 3097.0, 2475.0, 1089.0, 2442.0, 1290.0, 1216.0, 2003.0, 481.0, 45174.0, 2202.0, 3084.0, 1671.0, 2180.0, 1645.0, 673.0, 1105.0, 3291.0, 2511.0, 2209.0, 1170.0, 2493.0, 2986.0, 1214.0, 1631.0, 2343.0, 1510.0, 2368.0, 2667.0, 151.0, 2317.0, 700.0, 2893.0, 2464.0, 4321.0, 3191.0, 2749.0, 2530.0, 1771.0, 1324.0]
45174.0 128.0


In [38]:
from collections import defaultdict

cnt = defaultdict(int)
label_1 = list()
label_2 = list()
label_3andmore = list()
with open(path_csv + '/data/train_anno.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])
  for i,each in enumerate(train):
    # print(each)
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    cnt[len(each['labels'])] += 1
    if len(each['labels']) == 1:
      label_1.append(each)
    elif len(each['labels']) == 2:
      label_2.append(each)
    else:
      label_3andmore.append(each)

import json
for name, j in zip(['train_anno_1', 'train_anno_2', 'train_anno_3andmore'], [label_1, label_2, label_3andmore]):
  with open(path_csv + '/data/'+'{}.json'.format(name), 'w') as f:
    json.dump(j, f)
# label_1 = json.dumps(json)


82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}


In [50]:
  import pickle
  with open('coco_lt_adj.pkl', 'wb') as f:
    pickle.dump(di, f)
  np.save('coco_class_num.npy', gt_labels.sum(axis=0))

  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

In [64]:
with open("data/coco/img_id.pkl", "rb") as f:
  # data= pickle.load(f)
  data = f.readlines()
  data = [int(each) for each in data]
  # print(data)
  train_anno = []
  with open('data/coco/data/train_anno.json') as f:

    train = json.load(f)
    print(len(train), train[0])
    for each in train:
      img_id = int(each['file_name'].split('.')[0].split('_')[-1])
      if img_id in data:
        train_anno.append(each)
  print(len(train_anno))

  with open("data/coco/data/train_anno_LT.json", "w") as f:
    json.dump(train_anno, f)
  


82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
1861
